### Download all dependencies  

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.18.1-py_0 conda-forge

geographiclib- 100% |################################| Time: 0:00:00  23.08 MB/s
geopy-1.18.1-p 100% |################################| Time: 0:00:00  36.33 MB/s
Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00  54.39 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  32.89 MB/s
vincent-0.4.4- 100% |###################

### Download and explore dataset  

In [2]:
# We use the link https://geo.nyu.edu/catalog/nyu_2451_34572 to explore neighborhoods in New York
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Data downloaded!


In [3]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [4]:
# Define a new variable that includes all relevant data in the features key
neighborhoods_data = newyork_data['features']

In [5]:
# Let's take a look at first item in this list
neighborhoods_data[0]

{'geometry': {'coordinates': [-73.84720052054902, 40.89470517661],
  'type': 'Point'},
 'geometry_name': 'geom',
 'id': 'nyu_2451_34572.1',
 'properties': {'annoangle': 0.0,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661],
  'borough': 'Bronx',
  'name': 'Wakefield',
  'stacked': 1},
 'type': 'Feature'}

### Transform the data into a panda dataframe 

In [6]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude']

# instantiate the dataframe 
neighborhoods = pd.DataFrame(columns=column_names)

In [7]:
neighborhoods

,Borough,Neighborhood,Latitude,Longitude


In [8]:
# Let's loop through the data and fill the dataframe one row at a time
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough']
    neighborhood_name = data['properties']['name']
    
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                         'Neighborhood': neighborhood_name,
                                         'Latitude': neighborhood_lat,
                                         'Longitude': neighborhood_lon}, ignore_index=True)

In [9]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


### Let's focus on venues in Manhattan only due to time constraint of our tours

In [10]:
# Let's segment and cluster only the neighborhoods in Manhattan

manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


In [11]:
# Let's get the geographical coordinates of Manhattan
address = 'Manhattan, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinates of Manhattan are {}, {}.'.format(latitude, longitude))

The geographical coordinates of Manhattan are 40.7900869, -73.9598295.


In [12]:
# Let's visualize Manhattan with the neighborhoods in it
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map 
for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
         [lat, lng],
         radius=5,
         popup=label,
         color='blue',
         fill=True,
         fill_color='#3186cc',
         fill_opacity=0.7,
         parse_html=False).add_to(map_manhattan)
    
map_manhattan

### Utilize Foursqaure API to explore and segment neighborhoods in Manhattan

In [13]:
CLIENT_ID = 'UKUQMLW3PTAGNUKGDGG35OUF2WDZ05UGJRAWQPIKAVYVM5YG'
CLIENT_SECRET = 'RLVZEYK4G3PHEES5EZRFP1CKT3J3NKGCUJ4SQQWTHMVWKCHB'
VERSION = '20190222'
LIMIT = 100
Radius = 500

In [14]:
# Let's create a function to explore all neighborhoods in Manhattan

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [15]:
manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                  latitudes=manhattan_data['Latitude'],
                                  longitudes=manhattan_data['Longitude']
                                  )

Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


In [16]:
manhattan_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Battery Park City,100,100,100,100,100,100
Carnegie Hill,100,100,100,100,100,100
Central Harlem,44,44,44,44,44,44
Chelsea,100,100,100,100,100,100
Chinatown,100,100,100,100,100,100
Civic Center,100,100,100,100,100,100
Clinton,100,100,100,100,100,100
East Harlem,43,43,43,43,43,43
East Village,100,100,100,100,100,100


### Analyze each neighborhood 

In [17]:
# one hot encoding
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Cafeteria,Café,Cambodian Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Bookstore,College Cafeteria,College Gym,College Theater,Comedy Club,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Nightlife,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Public Art,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shopping Mall,Skate Park,Smoke Shop,Snack Place,Soba Restaurant,Social Club,Soup Place,South Indian Restaurant,Southe

### Group rows by neighborhood and taking the mean of frequency of occurence of each category 

In [18]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Cafeteria,Café,Cambodian Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Bookstore,College Cafeteria,College Gym,College Theater,Comedy Club,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Nightlife,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Public Art,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shopping Mall,Skate Park,Smoke Shop,Snack Place,Soba Restaurant,Social Club,Soup Place,South Indian Restaurant,Southe

### Print each neighborhood with the top 5 most common venues 

In [19]:
num_top_venues = 5

for hood in manhattan_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = manhattan_grouped[manhattan_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns =['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq':2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Battery Park City----
                venue  freq
0                Park  0.08
1         Coffee Shop  0.08
2               Hotel  0.05
3  Italian Restaurant  0.03
4           Wine Shop  0.03


----Carnegie Hill----
            venue  freq
0     Pizza Place  0.06
1            Café  0.05
2  Cosmetics Shop  0.05
3     Coffee Shop  0.05
4             Bar  0.03


----Central Harlem----
                 venue  freq
0   African Restaurant  0.07
1   Chinese Restaurant  0.05
2   Seafood Restaurant  0.05
3  American Restaurant  0.05
4    French Restaurant  0.05


----Chelsea----
                 venue  freq
0          Coffee Shop  0.07
1   Italian Restaurant  0.06
2       Ice Cream Shop  0.05
3  American Restaurant  0.04
4               Bakery  0.04


----Chinatown----
                   venue  freq
0     Chinese Restaurant  0.09
1     Dim Sum Restaurant  0.04
2    American Restaurant  0.04
3           Cocktail Bar  0.04
4  Vietnamese Restaurant  0.04


----Civic Center----
                  

### Let's put that into dataframe 

In [20]:
# Write a function to sort the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [21]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Coffee Shop,Park,Hotel,Gym,Food Truck,Italian Restaurant,Wine Shop,Sandwich Place,Clothing Store,Memorial Site
1,Carnegie Hill,Pizza Place,Cosmetics Shop,Coffee Shop,Café,Japanese Restaurant,Gym,French Restaurant,Yoga Studio,Bookstore,Bar
2,Central Harlem,African Restaurant,Gym / Fitness Center,Seafood Restaurant,Public Art,French Restaurant,American Restaurant,Chinese Restaurant,Art Gallery,Southern / Soul Food Restaurant,Spa
3,Chelsea,Coffee Shop,Italian Restaurant,Ice Cream Shop,Bakery,Nightclub,American Restaurant,Hotel,Seafood Restaurant,Theater,Health & Beauty Service
4,Chinatown,Chinese Restaurant,Cocktail Bar,Dim Sum Restaurant,American Restaurant,Vietnamese Restaurant,Bubble Tea Shop,Noodle House,Salon / Barbershop,Hotpot Restaurant,Bakery


### Cluster Neighborhoods

In [22]:
# set number of clusters 
kclusters = 5

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering 
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe 
kmeans.labels_[0:10]

array([2, 4, 4, 2, 4, 4, 2, 3, 4, 2], dtype=int32)

In [23]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

manhattan_merged = manhattan_data

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,2,Coffee Shop,Discount Store,Clothing Store,Big Box Store,Supplement Shop,Steakhouse,Spa,Shoe Store,Seafood Restaurant,Sandwich Place
1,Manhattan,Chinatown,40.715618,-73.994279,4,Chinese Restaurant,Cocktail Bar,Dim Sum Restaurant,American Restaurant,Vietnamese Restaurant,Bubble Tea Shop,Noodle House,Salon / Barbershop,Hotpot Restaurant,Bakery
2,Manhattan,Washington Heights,40.851903,-73.936900,4,Café,Bakery,Mobile Phone Shop,New American Restaurant,Mexican Restaurant,Park,Latin American Restaurant,Spanish Restaurant,Tapas Restaurant,Gym
3,Manhattan,Inwood,40.867684,-73.921210,3,Lounge,Mexican Restaurant,Café,Pizza Place,Restaurant,Frozen Yogurt Shop,Spanish Restaurant,Chinese Restaurant,Bakery,Park
4,Manhattan,Hamilton Heights,40.823604,-73.949688,3,Mexican Restaurant,Coffee Shop,Deli / Bodega,Café,Pizza Place,Cocktail Bar,Indian Restaurant,Sandwich Place,Liquor Store,Yoga Studio


In [24]:
#create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine clusters 

In [25]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,Stuyvesant Town,Bar,Park,German Restaurant,Harbor / Marina,Cocktail Bar,Coffee Shop,Heliport,Farmers Market,Pet Service,Basketball Court


In [26]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
33,Midtown South,Korean Restaurant,Hotel Bar,Hotel,Cosmetics Shop,Japanese Restaurant,Italian Restaurant,Bakery,Boutique,Cocktail Bar,Coffee Shop


In [27]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 2, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Marble Hill,Coffee Shop,Discount Store,Clothing Store,Big Box Store,Supplement Shop,Steakhouse,Spa,Shoe Store,Seafood Restaurant,Sandwich Place
5,Manhattanville,Italian Restaurant,Park,Mexican Restaurant,Coffee Shop,Seafood Restaurant,Cuban Restaurant,Ramen Restaurant,Bike Trail,Sushi Restaurant,Supermarket
9,Yorkville,Italian Restaurant,Bar,Gym,Coffee Shop,Pizza Place,Sushi Restaurant,Deli / Bodega,Diner,Mexican Restaurant,Japanese Restaurant
10,Lenox Hill,Italian Restaurant,Sushi Restaurant,Coffee Shop,Pizza Place,Gym / Fitness Center,Sporting Goods Shop,Burger Joint,Gym,Turkish Restaurant,Thai Restaurant
11,Roosevelt Island,Deli / Bodega,Sandwich Place,Coffee Shop,Gym,Park,Bus Station,Supermarket,Farmers Market,Metro Station,Outdoors & Recreation
13,Lincoln Square,Gym / Fitness Center,Theater,Italian Restaurant,Plaza,Café,Concert Hall,French Restaurant,Opera House,Park,Performing Arts Venue
14,Clinton,Theater,Gym / Fitness Center,American Restaurant,Hotel,Gym,Italian Restaurant,Spa,Wine Shop,Lounge,Food Court
15,Midtown,Clothing Store,Hotel,Steakhouse,Cocktail Bar,Theater,Food Truck,American Restaurant,Bakery,Bookstore,Spa
16,Murray Hill,Hotel,Coffee Shop,Italian Restaurant,Japanese Restaurant,Gym,Sandwich Place,Bar,American Restaurant,Spa,French Restaurant
17,Chelsea,Coffee Shop,Italian Restaurant,Ice Cream Shop,Bakery,Nightclub,American Restaurant,Hotel,Seafood Restaurant,Theater,Health & Beauty Service


In [28]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 3, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Inwood,Lounge,Mexican Restaurant,Café,Pizza Place,Restaurant,Frozen Yogurt Shop,Spanish Restaurant,Chinese Restaurant,Bakery,Park
4,Hamilton Heights,Mexican Restaurant,Coffee Shop,Deli / Bodega,Café,Pizza Place,Cocktail Bar,Indian Restaurant,Sandwich Place,Liquor Store,Yoga Studio
7,East Harlem,Mexican Restaurant,Bakery,Deli / Bodega,Latin American Restaurant,Thai Restaurant,Convenience Store,Café,Taco Place,Street Art,Steakhouse
36,Tudor City,Mexican Restaurant,Park,Sushi Restaurant,Café,Pizza Place,Asian Restaurant,Hotel,Greek Restaurant,Deli / Bodega,Diner


### Let's explore Midtown neighborhood 

In [30]:
# Let's segment and cluster only the neighborhoods in Manhattan

midtown_data = manhattan_data[manhattan_data['Neighborhood'] == 'Midtown'].reset_index(drop=True)
midtown_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Midtown,40.754691,-73.981669


In [40]:
# Let's create a function to explore all neighborhoods in Manhattan

def getNearbyVenues1(names, latitudes, longitudes, radius=500):
    
    venues_list1=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results1 = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list1.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results1])

    nearby_venues1 = pd.DataFrame([item for venue_list1 in venues_list1 for item in venue_list1])
    nearby_venues1.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues1)

In [41]:
midtown_venues = getNearbyVenues1(names=midtown_data['Neighborhood'],
                                  latitudes=midtown_data['Latitude'],
                                  longitudes=midtown_data['Longitude']
                                  )

Midtown


In [43]:
# one hot encoding
midtown_onehot = pd.get_dummies(midtown_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
midtown_onehot['Neighborhood'] = midtown_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [midtown_onehot.columns[-1]] + list(midtown_onehot.columns[:-1])
midtown_onehot = midtown_onehot[fixed_columns]

midtown_onehot.head()

,Neighborhood,American Restaurant,Art Gallery,Bakery,Bookstore,Boxing Gym,Brazilian Restaurant,Burrito Place,Café,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Cosmetics Shop,Cuban Restaurant,Cycle Studio,Deli / Bodega,Fast Food Restaurant,Food Truck,French Restaurant,General College & University,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hawaiian Restaurant,Historic Site,Hotel,Indian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Massage Studio,Mediterranean Restaurant,Miscellaneous Shop,Optical Shop,Park,Pilates Studio,Pizza Place,Ramen Restaurant,Salad Place,Sandwich Place,Shoe Store,Smoke Shop,Spa,Sporting Goods Shop,Steakhouse,Sushi Restaurant,Szechuan Restaurant,Tailor Shop,Theater,Theme Park Ride / Attraction,Women's Store,Yoga Studio
0,Midtown,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Midtown,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Midtown,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,Midtown,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Midtown,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [67]:
midtown_grouped = midtown_onehot.groupby('Neighborhood').mean().reset_index()
midtown_grouped

,Neighborhood,American Restaurant,Art Gallery,Bakery,Bookstore,Boxing Gym,Brazilian Restaurant,Burrito Place,Café,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Cosmetics Shop,Cuban Restaurant,Cycle Studio,Deli / Bodega,Fast Food Restaurant,Food Truck,French Restaurant,General College & University,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hawaiian Restaurant,Historic Site,Hotel,Indian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Massage Studio,Mediterranean Restaurant,Miscellaneous Shop,Optical Shop,Park,Pilates Studio,Pizza Place,Ramen Restaurant,Salad Place,Sandwich Place,Shoe Store,Smoke Shop,Spa,Sporting Goods Shop,Steakhouse,Sushi Restaurant,Szechuan Restaurant,Tailor Shop,Theater,Theme Park Ride / Attraction,Women's Store,Yoga Studio
0,Midtown,0.03,0.01,0.03,0.03,0.01,0.01,0.01,0.02,0.02,0.01,0.07,0.04,0.03,0.01,0.02,0.01,0.02,0.01,0.01,0.03,0.02,0.01,0.01,0.01,0.01,0.02,0.01,0.01,0.01,0.06,0.01,0.02,0.01,0.01,0.02,0.01,0.01,0.01,0.02,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.03,0.02,0.04,0.02,0.01,0.02,0.04,0.01,0.01,0.01


In [68]:
# Write a function to sort the venues in descending order
def return_most_common_venues1(row, num_top_venues):
    row_categories1 = row.iloc[1:]
    row_categories_sorted1 = row_categories1.sort_values(ascending=False)
    
    return row_categories_sorted1.index.values[0:num_top_venues]

In [74]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind1 in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind1+1, indicators[ind1]))
    except:
        columns.append('{}th Most Common Venue'.format(ind1+1))

# create a new dataframe
midtown_venues_sorted = pd.DataFrame(columns=columns)
midtown_venues_sorted['Neighborhood'] = midtown_grouped['Neighborhood']

for ind1 in np.arange(manhattan_grouped.shape[0]):
    midtown_venues_sorted.iloc[ind1, 1:] = return_most_common_venues1(midtown_grouped.iloc[ind1, :], num_top_venues)

midtown_venues_sorted.head()

IndexError: single positional indexer is out-of-bounds

In [70]:
# set number of clusters 
kclusters = 1

midtown_grouped_clustering = midtown_grouped.drop('Neighborhood', 1)

# run k-means clustering 
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(midtown_grouped_clustering)

# check cluster labels generated for each row in the dataframe 
kmeans.labels_[0:10]

array([0], dtype=int32)

In [71]:
midtown_venues

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Midtown,40.754691,-73.981669,Bryant Park,40.753621,-73.983265,Park
1,Midtown,40.754691,-73.981669,New York Public Library Terrace,40.753017,-73.981480,Park
2,Midtown,40.754691,-73.981669,Nat Sherman Townhouse,40.753283,-73.980358,Smoke Shop
3,Midtown,40.754691,-73.981669,sweetgreen,40.754640,-73.983102,Salad Place
4,Midtown,40.754691,-73.981669,Sophie's Cuban Cuisine,40.755927,-73.980598,Cuban Restaurant
5,Midtown,40.754691,-73.981669,Xi'an Famous Foods,40.755926,-73.980751,Chinese Restaurant
6,Midtown,40.754691,-73.981669,Sofitel New York,40.755787,-73.981762,Hotel
7,Midtown,40.754691,-73.981669,Equinox East 43rd Street,40.754089,-73.979900,Gym
8,Midtown,40.754691,-73.981669,Gabriel Kreuther,40.754143,-73.983505,French Restaurant
9,Midtown,40.754691,-73.981669,Harvard Club of New York City,40.755485,-73.981502,Historic Site


In [72]:
midtown_grouped.head()

,Neighborhood,American Restaurant,Art Gallery,Bakery,Bookstore,Boxing Gym,Brazilian Restaurant,Burrito Place,Café,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Cosmetics Shop,Cuban Restaurant,Cycle Studio,Deli / Bodega,Fast Food Restaurant,Food Truck,French Restaurant,General College & University,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hawaiian Restaurant,Historic Site,Hotel,Indian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Massage Studio,Mediterranean Restaurant,Miscellaneous Shop,Optical Shop,Park,Pilates Studio,Pizza Place,Ramen Restaurant,Salad Place,Sandwich Place,Shoe Store,Smoke Shop,Spa,Sporting Goods Shop,Steakhouse,Sushi Restaurant,Szechuan Restaurant,Tailor Shop,Theater,Theme Park Ride / Attraction,Women's Store,Yoga Studio
0,Midtown,0.03,0.01,0.03,0.03,0.01,0.01,0.01,0.02,0.02,0.01,0.07,0.04,0.03,0.01,0.02,0.01,0.02,0.01,0.01,0.03,0.02,0.01,0.01,0.01,0.01,0.02,0.01,0.01,0.01,0.06,0.01,0.02,0.01,0.01,0.02,0.01,0.01,0.01,0.02,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.03,0.02,0.04,0.02,0.01,0.02,0.04,0.01,0.01,0.01


In [73]:
midtown_merged.head()

NameError: name 'midtown_merged' is not defined

In [59]:
manhattan_merged.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,2,Coffee Shop,Discount Store,Clothing Store,Big Box Store,Supplement Shop,Steakhouse,Spa,Shoe Store,Seafood Restaurant,Sandwich Place
1,Manhattan,Chinatown,40.715618,-73.994279,4,Chinese Restaurant,Cocktail Bar,Dim Sum Restaurant,American Restaurant,Vietnamese Restaurant,Bubble Tea Shop,Noodle House,Salon / Barbershop,Hotpot Restaurant,Bakery
2,Manhattan,Washington Heights,40.851903,-73.936900,4,Café,Bakery,Mobile Phone Shop,New American Restaurant,Mexican Restaurant,Park,Latin American Restaurant,Spanish Restaurant,Tapas Restaurant,Gym
3,Manhattan,Inwood,40.867684,-73.921210,3,Lounge,Mexican Restaurant,Café,Pizza Place,Restaurant,Frozen Yogurt Shop,Spanish Restaurant,Chinese Restaurant,Bakery,Park
4,Manhattan,Hamilton Heights,40.823604,-73.949688,3,Mexican Restaurant,Coffee Shop,Deli / Bodega,Café,Pizza Place,Cocktail Bar,Indian Restaurant,Sandwich Place,Liquor Store,Yoga Studio


In [65]:
#create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(midtown_venues['Latitude'], midtown_venues['Longitude'], midtown_venues['Neighborhood'], midtown_bv['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

KeyError: 'Latitude'

In [66]:
midtown_merged.head()

NameError: name 'midtown_merged' is not defined